In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import tqdm
import random
import glob
import json

In [2]:
RANDOM_SEED=42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [3]:
ZAPPOS_ROOT = '/home/amrith/zappos-50k/'
ZAPPOS_IMAGES_ROOT = '/home/amrith/zappos-50k/ut-zap50k-images-square'

In [4]:
df = pd.read_csv(f'{ZAPPOS_ROOT}/ut-zap50k-data/meta-data-bin.csv')
df['HeelHeight.High.heel'] = df['HeelHeight.4in...4.3.4in'] + df['HeelHeight.5in...over']
df['HeelHeight.Short.heel'] = df['HeelHeight.Flat'] + df['HeelHeight.Under.1in'] + df['HeelHeight.1in...1.3.4in']

In [5]:
df.shape

(50025, 154)

In [6]:
print(sorted(df.columns))

['CID', 'Category.Boots', 'Category.Sandals', 'Category.Shoes', 'Category.Slippers', 'Closure.Adjustable', 'Closure.Ankle.Strap', 'Closure.Ankle.Wrap', 'Closure.Belt', 'Closure.Buckle', 'Closure.Bungee', 'Closure.Button.Loop', 'Closure.Elastic.Gore', 'Closure.Hook.and.Loop', 'Closure.Lace.up', 'Closure.Monk.Strap', 'Closure.Pull.on', 'Closure.Sling.Back', 'Closure.Slip.On', 'Closure.Snap', 'Closure.Spat.Strap', 'Closure.T.Strap', 'Closure.Toggle', 'Closure.Zipper', 'Gender.Boys', 'Gender.Girls', 'Gender.Men', 'Gender.Women', 'HeelHeight.1in...1.3.4in', 'HeelHeight.2in...2.3.4in', 'HeelHeight.3in...3.3.4in', 'HeelHeight.4in...4.3.4in', 'HeelHeight.5in...over', 'HeelHeight.Flat', 'HeelHeight.High.heel', 'HeelHeight.Short.heel', 'HeelHeight.Under.1in', 'Insole.EVA', 'Insole.Gel', 'Insole.Hypoallergenic', 'Insole.Latex.Lined', 'Insole.Leather', 'Insole.Memory.Foam', 'Insole.Moisture.Wicking', 'Insole.Orthotic.Friendly', 'Insole.Padded', 'Insole.Polyurethane', 'Insole.Poron', 'Insole.Remova

In [7]:
# Attribute list
ATTR_LIST_1 = ['Category.Shoes', 'Category.Sandals', 'SubCategory.Oxfords', 'SubCategory.Heel', 'SubCategory.Boot', 'SubCategory.Slipper.Flats', 'SubCategory.Flats', 'SubCategory.Slipper.Heels', 'SubCategory.Athletic', 'SubCategory.Knee.High', 'SubCategory.Crib.Shoes', 'SubCategory.Over.the.Knee', 'HeelHeight.High.heel', 'HeelHeight.Short.heel', 'Closure.Pull.on', 'Closure.Ankle.Strap', 'Closure.Zipper', 'Closure.Elastic.Gore', 'Closure.Sling.Back', 'Closure.Toggle', 'Closure.Snap', 'Closure.T.Strap', 'Closure.Spat.Strap', 'Gender.Men', 'Gender.Boys', 'Material.Rubber', 'Material.Wool', 'Material.Silk', 'Material.Aluminum', 'Material.Plastic', 'ToeStyle.Capped Toe', 'ToeStyle.Square Toe', 'ToeStyle.Snub Toe', 'ToeStyle.Bicycle Toe', 'ToeStyle.Open Toe', 'ToeStyle.Pointed Toe', 'ToeStyle.Almond', 'ToeStyle.Apron Toe', 'ToeStyle.Snip Toe', 'ToeStyle.Medallion']
ATTR_LIST_2 = ['Category.Boots', 'Category.Slippers', 'SubCategory.Mid.Calf', 'SubCategory.Ankle', 'SubCategory.Loafers', 'SubCategory.Boat.Shoes', 'SubCategory.Clogs.and.Mules', 'SubCategory.Sneakers.and.Athletic.Shoes', 'SubCategory.Heels', 'SubCategory.Prewalker', 'SubCategory.Prewalker.Boots', 'SubCategory.Firstwalker', 'Closure.Lace.up', 'Closure.Buckle', 'Closure.Hook.and.Loop', 'Closure.Slip.On', 'Closure.Ankle.Wrap', 'Closure.Bungee', 'Closure.Adjustable',  'Closure.Button.Loop', 'Closure.Monk.Strap', 'Closure.Belt', 'Gender.Women', 'Gender.Girls', 'Material.Suede', 'Material.Snakeskin', 'Material.Corduroy', 'Material.Horse.Hair', 'Material.Stingray', 'ToeStyle.Round Toe', 'ToeStyle.Closed Toe', 'ToeStyle.Moc Toe', 'ToeStyle.Wingtip', 'ToeStyle.Center Seam', 'ToeStyle.Algonquin', 'ToeStyle.Bump Toe', 'ToeStyle.Wide Toe Box', 'ToeStyle.Peep Toe']
FULL_ATTR_LIST = sorted(list(set(ATTR_LIST_1 + ATTR_LIST_2)))
for x in FULL_ATTR_LIST:
    if x not in df.columns: 
        print(x)
print(len(FULL_ATTR_LIST))

78


In [8]:
ATTR_LIST = []
MIN_IMAGES_PER_ATTR = 1000
for a, x in zip(FULL_ATTR_LIST, df.loc[:, FULL_ATTR_LIST].apply(sum, axis=0)):
    if x > MIN_IMAGES_PER_ATTR:
        ATTR_LIST.append(a)
print(len(ATTR_LIST))

38


In [9]:
ATTR_LIST

['Category.Boots',
 'Category.Sandals',
 'Category.Shoes',
 'Category.Slippers',
 'Closure.Ankle.Strap',
 'Closure.Buckle',
 'Closure.Elastic.Gore',
 'Closure.Hook.and.Loop',
 'Closure.Lace.up',
 'Closure.Pull.on',
 'Closure.Slip.On',
 'Closure.Zipper',
 'Gender.Boys',
 'Gender.Girls',
 'Gender.Men',
 'Gender.Women',
 'HeelHeight.High.heel',
 'HeelHeight.Short.heel',
 'Material.Rubber',
 'Material.Suede',
 'SubCategory.Ankle',
 'SubCategory.Clogs.and.Mules',
 'SubCategory.Flats',
 'SubCategory.Heels',
 'SubCategory.Knee.High',
 'SubCategory.Loafers',
 'SubCategory.Mid.Calf',
 'SubCategory.Oxfords',
 'SubCategory.Slipper.Flats',
 'SubCategory.Sneakers.and.Athletic.Shoes',
 'ToeStyle.Almond',
 'ToeStyle.Capped Toe',
 'ToeStyle.Closed Toe',
 'ToeStyle.Moc Toe',
 'ToeStyle.Open Toe',
 'ToeStyle.Peep Toe',
 'ToeStyle.Pointed Toe',
 'ToeStyle.Round Toe']

1. logic to sample attribute list
2. logic to check if attributes are from same category
3. logic to sample images that are positive for the attribute pair

In [10]:
def is_same_category(attr_pair):
    return len(set([attr.lower().split(".")[0] for attr in attrs])) < len(attr_pair)
    
def sample_attrs(attributes_list, n_attrs=2):
    return np.random.choice(attributes_list, n_attrs, replace=False)

def get_all_positive_images(df, attrs):
    return df[attrs].apply(sum, axis=1) == len(attrs)

def get_all_negative_images(df, attrs):
    return (1-df[attrs]).apply(sum, axis=1) == len(attrs)

def sample_images(df, selected_images, n_samples):
    return list(np.random.choice(df[selected_images]["CID"].values, n_samples, replace=False))

In [11]:
N_SAMPLES = 8000
POSITIVE_IMAGES_THRESHOLD = 20
NEGATIVE_IMAGES_THRESHOLD = 20
N_SUPPORT = 10
N_QUERY = 10

In [12]:
CID_to_impath = {x.split("/")[-1][:-4].replace(".", "-"):x for x in glob.glob(f"{ZAPPOS_IMAGES_ROOT}/**/*.jpg", recursive=True)}

In [ ]:
# main logic for sampling
n_selected = 0
dataset = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
counter = 0
import time
st = time.time()
while n_selected < N_SAMPLES:
    counter += 1
    # get attributes
    attrs = sample_attrs(ATTR_LIST)
    print(attrs)
    # if any pair of the sampled attributes are from the same category then reject
    if is_same_category(attrs):
        # print("Dropped -- same category", attrs)
        continue
    # get images which satisfy the attributes
    positive_images = get_all_positive_images(df, attrs)
    negative_images = get_all_negative_images(df, attrs)
    # negative_images = 1 - positive_images
    # if we want to enforce 0s for all attributes then we need to change the logic above
    # to include get_all_negative_images(df, attrs)
    
    # if the no of images that are positive for the chosen attributes is too less, then reject 
    if (sum(positive_images) < POSITIVE_IMAGES_THRESHOLD) or (sum(negative_images) < NEGATIVE_IMAGES_THRESHOLD) :
        # print("Dropped -- didnt meet threshold", attrs)
        continue
    
    positive = [CID_to_impath[z] for z in sample_images(df, positive_images, N_SUPPORT+N_QUERY)]
    negative = [CID_to_impath[z] for z in sample_images(df, negative_images, N_SUPPORT+N_QUERY)]
    
    dataset[n_selected][0]['support'] = negative[:N_SUPPORT]
    dataset[n_selected][0]['query'] = negative[N_SUPPORT:]
    
    dataset[n_selected][1]['support'] = positive[:N_SUPPORT]
    dataset[n_selected][1]['query'] = positive[N_SUPPORT:]
    
    dataset[n_selected][0]['attributes'] = ";".join(["!"+a for a in attrs])
    dataset[n_selected][1]['attributes'] = ";".join(attrs)
    
    
    n_selected += 1
    print(f'Selected {n_selected}/{counter} Time Elapsed {time.time()-st}')
        

['ToeStyle.Moc Toe' 'ToeStyle.Pointed Toe']
['Gender.Women' 'SubCategory.Clogs.and.Mules']
Selected 1/2 Time Elapsed 1.3562486171722412
['Material.Suede' 'ToeStyle.Open Toe']
Selected 2/3 Time Elapsed 2.699401378631592
['ToeStyle.Open Toe' 'Closure.Zipper']
Selected 3/4 Time Elapsed 4.025339841842651
['Closure.Lace.up' 'Gender.Women']
Selected 4/5 Time Elapsed 5.37087607383728
['Category.Sandals' 'SubCategory.Mid.Calf']
['Category.Boots' 'SubCategory.Heels']
['SubCategory.Sneakers.and.Athletic.Shoes' 'ToeStyle.Open Toe']
['SubCategory.Loafers' 'ToeStyle.Moc Toe']
Selected 5/9 Time Elapsed 10.648598194122314
['ToeStyle.Round Toe' 'ToeStyle.Closed Toe']
['Gender.Men' 'Material.Rubber']
Selected 6/11 Time Elapsed 12.001996994018555
['ToeStyle.Capped Toe' 'SubCategory.Loafers']
Selected 7/12 Time Elapsed 13.334845066070557
['SubCategory.Flats' 'SubCategory.Clogs.and.Mules']
['Closure.Elastic.Gore' 'Category.Sandals']
Selected 8/14 Time Elapsed 14.671065330505371
['SubCategory.Flats' 'ToeSt

['HeelHeight.High.heel' 'Category.Boots']
Selected 73/113 Time Elapsed 121.5810296535492
['Material.Rubber' 'Category.Boots']
Selected 74/114 Time Elapsed 122.89737272262573
['HeelHeight.Short.heel' 'SubCategory.Sneakers.and.Athletic.Shoes']
Selected 75/115 Time Elapsed 124.25538516044617
['Closure.Ankle.Strap' 'Material.Rubber']
Selected 76/116 Time Elapsed 125.60057282447815
['SubCategory.Flats' 'SubCategory.Heels']
['SubCategory.Clogs.and.Mules' 'SubCategory.Heels']
['ToeStyle.Closed Toe' 'Closure.Elastic.Gore']
Selected 77/119 Time Elapsed 126.92270946502686
['Gender.Girls' 'ToeStyle.Pointed Toe']
Selected 78/120 Time Elapsed 128.23170280456543
['ToeStyle.Almond' 'Gender.Women']
Selected 79/121 Time Elapsed 129.56311750411987
['HeelHeight.Short.heel' 'ToeStyle.Open Toe']
Selected 80/122 Time Elapsed 130.90081405639648
['Gender.Boys' 'ToeStyle.Pointed Toe']
['Gender.Boys' 'Category.Slippers']
Selected 81/124 Time Elapsed 133.52714848518372
['SubCategory.Slipper.Flats' 'Closure.Zippe

['Closure.Lace.up' 'Category.Slippers']
['Closure.Ankle.Strap' 'SubCategory.Flats']
Selected 137/229 Time Elapsed 247.53803873062134
['SubCategory.Ankle' 'SubCategory.Heels']
['ToeStyle.Round Toe' 'SubCategory.Ankle']
Selected 138/231 Time Elapsed 248.8648030757904
['SubCategory.Clogs.and.Mules' 'SubCategory.Slipper.Flats']
['Closure.Buckle' 'SubCategory.Knee.High']
Selected 139/233 Time Elapsed 250.203289270401
['Closure.Slip.On' 'Closure.Hook.and.Loop']
['Category.Sandals' 'Category.Slippers']
['Closure.Elastic.Gore' 'Category.Boots']
Selected 140/236 Time Elapsed 251.62583422660828
['SubCategory.Oxfords' 'Closure.Elastic.Gore']
['Gender.Boys' 'SubCategory.Loafers']
Selected 141/238 Time Elapsed 254.37886309623718
['HeelHeight.High.heel' 'Closure.Buckle']
Selected 142/239 Time Elapsed 255.7671399116516
['Material.Suede' 'Gender.Girls']
Selected 143/240 Time Elapsed 257.11944007873535
['Closure.Pull.on' 'Closure.Lace.up']
['SubCategory.Slipper.Flats' 'ToeStyle.Closed Toe']
Selected 14

Selected 254/420 Time Elapsed 452.8888273239136
['ToeStyle.Open Toe' 'SubCategory.Knee.High']
['SubCategory.Slipper.Flats' 'ToeStyle.Peep Toe']
['SubCategory.Mid.Calf' 'ToeStyle.Pointed Toe']
Selected 255/423 Time Elapsed 456.8365087509155
['ToeStyle.Open Toe' 'ToeStyle.Peep Toe']
['Closure.Slip.On' 'Closure.Lace.up']
['SubCategory.Knee.High' 'ToeStyle.Peep Toe']
['SubCategory.Sneakers.and.Athletic.Shoes' 'SubCategory.Knee.High']
['Closure.Hook.and.Loop' 'HeelHeight.Short.heel']
Selected 256/428 Time Elapsed 459.4741702079773
['ToeStyle.Round Toe' 'Category.Sandals']
Selected 257/429 Time Elapsed 460.780953168869
['ToeStyle.Pointed Toe' 'Category.Boots']
Selected 258/430 Time Elapsed 462.1369616985321
['Material.Suede' 'ToeStyle.Peep Toe']
Selected 259/431 Time Elapsed 463.4642632007599
['Category.Boots' 'Category.Slippers']
['Material.Suede' 'SubCategory.Ankle']
Selected 260/433 Time Elapsed 464.82023787498474
['Gender.Women' 'SubCategory.Knee.High']
Selected 261/434 Time Elapsed 466.

Selected 325/531 Time Elapsed 567.5239133834839
['Gender.Girls' 'Category.Shoes']
Selected 326/532 Time Elapsed 568.8491539955139
['ToeStyle.Round Toe' 'Material.Rubber']
Selected 327/533 Time Elapsed 570.1955778598785
['Closure.Elastic.Gore' 'Gender.Men']
Selected 328/534 Time Elapsed 571.4943964481354
['Gender.Women' 'ToeStyle.Capped Toe']
Selected 329/535 Time Elapsed 572.8681628704071
['Category.Slippers' 'ToeStyle.Moc Toe']
Selected 330/536 Time Elapsed 574.2534091472626
['Category.Shoes' 'SubCategory.Mid.Calf']
['Category.Sandals' 'SubCategory.Slipper.Flats']
['SubCategory.Ankle' 'SubCategory.Heels']
['SubCategory.Sneakers.and.Athletic.Shoes' 'SubCategory.Clogs.and.Mules']
['HeelHeight.High.heel' 'Closure.Ankle.Strap']
Selected 331/541 Time Elapsed 578.2521712779999
['Closure.Hook.and.Loop' 'ToeStyle.Peep Toe']
['Category.Slippers' 'ToeStyle.Closed Toe']
Selected 332/543 Time Elapsed 580.8717296123505
['ToeStyle.Round Toe' 'Closure.Pull.on']
Selected 333/544 Time Elapsed 582.2050

...

Selected 4719/7482 Time Elapsed 9305.467262983322
['Closure.Buckle' 'SubCategory.Loafers']
Selected 4720/7483 Time Elapsed 9306.744488477707
['SubCategory.Clogs.and.Mules' 'Material.Rubber']
Selected 4721/7484 Time Elapsed 9308.018653154373
['HeelHeight.Short.heel' 'Closure.Zipper']
Selected 4722/7485 Time Elapsed 9309.32562160492
['Closure.Pull.on' 'Gender.Men']
Selected 4723/7486 Time Elapsed 9310.60651731491
['ToeStyle.Round Toe' 'Closure.Pull.on']
Selected 4724/7487 Time Elapsed 9311.908754348755
['ToeStyle.Capped Toe' 'Closure.Elastic.Gore']
Selected 4725/7488 Time Elapsed 9313.225525140762
['Closure.Zipper' 'HeelHeight.High.heel']
Selected 4726/7489 Time Elapsed 9314.519662857056
['Material.Rubber' 'SubCategory.Slipper.Flats']
Selected 4727/7490 Time Elapsed 9315.80605006218
['Closure.Hook.and.Loop' 'SubCategory.Flats']
Selected 4728/7491 Time Elapsed 9317.081276893616
['SubCategory.Heels' 'Closure.Buckle']
Selected 4729/7492 Time Elapsed 9318.35869193077
['Material.Suede' 'Categ

['HeelHeight.Short.heel' 'Closure.Zipper']
Selected 4788/7594 Time Elapsed 9417.365355491638
['Gender.Men' 'ToeStyle.Pointed Toe']
Selected 4789/7595 Time Elapsed 9418.636372566223
['Closure.Slip.On' 'SubCategory.Flats']
Selected 4790/7596 Time Elapsed 9419.907047986984
['SubCategory.Oxfords' 'Category.Sandals']
['Closure.Slip.On' 'SubCategory.Heels']
Selected 4791/7598 Time Elapsed 9422.460042238235
['Category.Slippers' 'Gender.Men']
Selected 4792/7599 Time Elapsed 9423.736341714859
['SubCategory.Flats' 'Closure.Pull.on']
['Material.Rubber' 'Closure.Slip.On']
Selected 4793/7601 Time Elapsed 9426.260926008224
['ToeStyle.Peep Toe' 'Category.Boots']
Selected 4794/7602 Time Elapsed 9427.523160219193
['SubCategory.Oxfords' 'Category.Slippers']
['ToeStyle.Open Toe' 'Category.Shoes']
Selected 4795/7604 Time Elapsed 9430.064072847366
['SubCategory.Loafers' 'SubCategory.Sneakers.and.Athletic.Shoes']
['ToeStyle.Almond' 'Gender.Girls']
['ToeStyle.Moc Toe' 'Material.Suede']
Selected 4796/7607 Tim

Selected 4857/7704 Time Elapsed 9531.130373001099
['Closure.Pull.on' 'Closure.Ankle.Strap']
['Category.Slippers' 'Closure.Lace.up']
['ToeStyle.Round Toe' 'Category.Slippers']
Selected 4858/7707 Time Elapsed 9533.628762245178
['Closure.Pull.on' 'Category.Boots']
Selected 4859/7708 Time Elapsed 9534.923486948013
['Closure.Elastic.Gore' 'Category.Slippers']
Selected 4860/7709 Time Elapsed 9536.222236394882
['Gender.Men' 'Closure.Buckle']
Selected 4861/7710 Time Elapsed 9537.482001304626
['SubCategory.Flats' 'SubCategory.Slipper.Flats']
['Closure.Elastic.Gore' 'SubCategory.Sneakers.and.Athletic.Shoes']
Selected 4862/7712 Time Elapsed 9538.7791492939
['Closure.Lace.up' 'SubCategory.Slipper.Flats']
['Material.Rubber' 'SubCategory.Slipper.Flats']
Selected 4863/7714 Time Elapsed 9541.305079221725
['Closure.Pull.on' 'SubCategory.Sneakers.and.Athletic.Shoes']
['ToeStyle.Peep Toe' 'SubCategory.Loafers']
['Closure.Ankle.Strap' 'SubCategory.Knee.High']
['SubCategory.Mid.Calf' 'Category.Shoes']
['Cl

Selected 4921/7821 Time Elapsed 9655.120154857635
['SubCategory.Heels' 'Gender.Girls']
Selected 4922/7822 Time Elapsed 9656.382175683975
['Material.Rubber' 'ToeStyle.Almond']
Selected 4923/7823 Time Elapsed 9657.64778637886
['SubCategory.Heels' 'Category.Boots']
['Category.Slippers' 'HeelHeight.High.heel']
['Closure.Zipper' 'Closure.Buckle']
['Gender.Girls' 'Gender.Men']
['SubCategory.Ankle' 'ToeStyle.Peep Toe']
Selected 4924/7828 Time Elapsed 9661.439934492111
['Closure.Hook.and.Loop' 'Material.Rubber']
Selected 4925/7829 Time Elapsed 9662.707428693771
['Closure.Zipper' 'SubCategory.Oxfords']
['Category.Slippers' 'HeelHeight.Short.heel']
Selected 4926/7831 Time Elapsed 9665.230908870697
['Closure.Lace.up' 'Material.Suede']
Selected 4927/7832 Time Elapsed 9666.531661987305
['SubCategory.Flats' 'Closure.Hook.and.Loop']
Selected 4928/7833 Time Elapsed 9667.80919289589
['SubCategory.Slipper.Flats' 'SubCategory.Loafers']
['Category.Boots' 'ToeStyle.Open Toe']
Selected 4929/7835 Time Elapse

['Gender.Women' 'ToeStyle.Round Toe']
Selected 4996/7924 Time Elapsed 9766.13853430748
['ToeStyle.Peep Toe' 'HeelHeight.Short.heel']
Selected 4997/7925 Time Elapsed 9767.376878261566
['Closure.Ankle.Strap' 'ToeStyle.Open Toe']
Selected 4998/7926 Time Elapsed 9768.628182172775
['Closure.Buckle' 'SubCategory.Slipper.Flats']
['SubCategory.Loafers' 'ToeStyle.Moc Toe']
Selected 4999/7928 Time Elapsed 9771.141308546066
['HeelHeight.Short.heel' 'Material.Rubber']
Selected 5000/7929 Time Elapsed 9772.404334783554
['SubCategory.Oxfords' 'ToeStyle.Pointed Toe']
['Category.Boots' 'Closure.Buckle']
Selected 5001/7931 Time Elapsed 9774.936666250229
['SubCategory.Flats' 'ToeStyle.Moc Toe']
Selected 5002/7932 Time Elapsed 9776.216812610626
['Category.Sandals' 'Gender.Girls']
Selected 5003/7933 Time Elapsed 9777.507595777512
['SubCategory.Flats' 'Category.Sandals']
['Closure.Zipper' 'Closure.Elastic.Gore']
['ToeStyle.Pointed Toe' 'Category.Slippers']
['ToeStyle.Moc Toe' 'Closure.Zipper']
Selected 5004

In [15]:
len(dataset)

8000

In [ ]:
json.dump(dataset, open(f'zappos-minImg{MIN_IMAGES_PER_ATTR}-nsamp{N_SAMPLES}-ns{N_SUPPORT}-nq{N_QUERY}.json', 'w'))